In [ ]:
# # This file is part of Theano Geometry
#
# Copyright (C) 2017, Stefan Sommer (sommer@di.ku.dk)
# https://bitbucket.org/stefansommer/theanogemetry
#
# Theano Geometry is free software: you can redistribute it and/or modify
# it under the terms of the GNU General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.
#
# Theano Geometry is distributed in the hope that it will be useful,
# but WITHOUT ANY WARRANTY; without even the implied warranty of
# MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
# GNU General Public License for more details.
#
# You should have received a copy of the GNU General Public License
# along with Theano Geometry. If not, see <http://www.gnu.org/licenses/>.
#

# Stochastic Lie group dynamics

In [ ]:
# SO(3)
from src.groups.SON import *
G = SON(3)
print(G)

# SO(3) acts on S^2
from src.manifolds.S2 import *
M = S2()
print(M)

from src.plotting import *

In [ ]:
# setup for testing different versions of stochastic dynamics
q = np.array([1e-6,0,0])
g = G.psif(q)
v = np.array([0,1,1])

from src.group import invariant_metric
invariant_metric.initialize(G)
p = G.sharppsif(q,v)
mu = G.sharpVf(v)
print(p)
print(mu)

from src.group import energy
energy.initialize(G)

In [ ]:
# Brownian motion
from src.stochastics import Brownian_inv
Brownian_inv.initialize(G)

# srng.seed(422)
srng.seed(422)
(ts,gsv) = G.Brownian_invf(g,dWsf(G.dim.eval()))
newfig()
G.plotg(gsv)
plt.show()
#plt.savefig('stocso3.pdf')

# on S2
newfig()
M.plot()
x = np.array([0,0,1])
M.plotx(M.actsf(gsv.transpose((1,2,0)),x).T)
plt.show()
#plt.savefig('stocso32.pdf')

In [ ]:
# Brownian processes
from src.stochastics import Brownian_process
Brownian_process.initialize(G)

(ts,gsv) = G.Brownian_processf(g,dWsf(G.dim.eval()))
newfig()
G.plotg(gsv)
plt.show()

In [ ]:
# Stochastic coadjoint motion
from src.stochastics import stochastic_coadjoint
stochastic_coadjoint.initialize(G)

(ts,musv) = G.stochastic_coadjointf(mu,dWsf(G.dim.eval()))
(ts,gsv) = G.stochastic_coadjointrecf(g,musv)
newfig()
G.plotg(gsv)
plt.show()

In [ ]:
# Delyon/Hu guided process
from src.stochastics.guided_process import *

# phi = lambda g,v: -(g-v)
phi = lambda g,v: T.tensordot(G.inv(G.sigma),G.LAtoV(G.log(G.invtrns(G.inv(g),v))),(1,0))
sde_Brownian_inv_guided = get_sde_guided(G.sde_Brownian_inv,phi,lambda q: G.sigma)
Brownian_inv_guided = lambda g,v,dWt: integrate_sde(sde_Brownian_inv_guided,
                                                   integrator_stratonovich,
                                                   g,dWt,T.constant(0.),T.constant(0.),v)
g0 = G.element()
v = G.element()
Brownian_inv_guidedf = theano.function([g0,v,dWt], Brownian_inv_guided(g0,v,dWt)[:4])

# derivatives
theta = G.sigma # parameters
def dlog_likelihood(g,v,dWt):
    s = Brownian_inv_guided(g,v,dWt)
    dlog_likelihood = T.grad(s[2][-1],G.theta)
    
    return [s[0],s[1],s[2],s[3],dlog_likelihood]
dlog_likelihoodf = theano.function([g0,v,dWt],dlog_likelihood(g0,v,dWt))

(ts,gsv,log_likelihood,log_varphi) = Brownian_inv_guidedf(g,G.psif(v),dWsf(G.dim.eval()))
print("log likelihood: ", log_likelihood[-1], ", log varphi: ", log_varphi[-1])
newfig()
G.plotg(gsv)
G.plotg(v,color='k')
plt.show()

# on S2
newfig()
M.plot()
M.plotx(actsf(gsv.transpose((1,2,0)),x0).T)
M.plotx(actf(v,x0),color='k')
plt.show()